In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np 
import pandas as pd
import os
import seaborn as sns
import math
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
df1=pd.read_csv('train.csv')
df2=pd.read_csv('test.csv')

In [3]:
df3=pd.read_csv('test.csv')

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625134 entries, 0 to 625133
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  625134 non-null  object 
 1   vendor_id           625134 non-null  int64  
 2   pickup_datetime     625134 non-null  object 
 3   passenger_count     625134 non-null  int64  
 4   pickup_longitude    625134 non-null  float64
 5   pickup_latitude     625134 non-null  float64
 6   dropoff_longitude   625134 non-null  float64
 7   dropoff_latitude    625134 non-null  float64
 8   store_and_fwd_flag  625134 non-null  object 
dtypes: float64(4), int64(2), object(3)
memory usage: 42.9+ MB


In [6]:
df1.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [7]:
df1= df1.loc[df1['trip_duration']<1000000]

In [8]:
#df1.drop('id',inplace=True,axis=1)
df1.drop('dropoff_datetime',inplace=True,axis=1)

In [9]:
df1["pickup_datetime"]=pd.to_datetime(df1["pickup_datetime"])

In [10]:
df1['day_of_week'] = df1['pickup_datetime'].dt.day_name()

In [11]:
df1['hour_of_the_day']=df1['pickup_datetime'].dt.hour
df1['month']=df1['pickup_datetime'].dt.month

In [12]:
df1['day_of_week']=df1['day_of_week'].map({'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7})
df1['store_and_fwd_flag']=df1['store_and_fwd_flag'].map({'N':0,'Y':1})
df1

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,day_of_week,hour_of_the_day,month
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,0,455,1,17,3
1,id2377394,1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,0,663,7,0,6
2,id3858529,2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,0,2124,2,11,1
3,id3504673,2,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,0,429,3,19,4
4,id2181028,2,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,0,435,6,13,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2,2016-04-08 13:31:04,4,-73.982201,40.745522,-73.994911,40.740170,0,778,5,13,4
1458640,id1049543,1,2016-01-10 07:35:15,1,-74.000946,40.747379,-73.970184,40.796547,0,655,7,7,1
1458641,id2304944,2,2016-04-22 06:57:41,1,-73.959129,40.768799,-74.004433,40.707371,0,764,5,6,4
1458642,id2714485,1,2016-01-05 15:56:26,1,-73.982079,40.749062,-73.974632,40.757107,0,373,2,15,1


In [13]:
#df2.drop('id',inplace=True,axis=1)
#df2.drop('dropoff_datetime',inplace=True,axis=1)

In [14]:
df2["pickup_datetime"]=pd.to_datetime(df2["pickup_datetime"])

In [15]:
df2['day_of_week'] = df2['pickup_datetime'].dt.day_name()

In [16]:
df2['hour_of_the_day']=df2['pickup_datetime'].dt.hour
df2['month']=df2['pickup_datetime'].dt.month

In [17]:
df2['day_of_week']=df2['day_of_week'].map({'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7})
df2['store_and_fwd_flag']=df2['store_and_fwd_flag'].map({'N':0,'Y':1})
df2

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,day_of_week,hour_of_the_day,month
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,0,4,23,6
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,0,4,23,6
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,0,4,23,6
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,0,4,23,6
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,0,4,23,6
...,...,...,...,...,...,...,...,...,...,...,...,...
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,0,5,0,1
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,0,5,0,1
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,0,5,0,1
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,0,5,0,1


In [18]:
def haversine(lat1, lon1, lat2, lon2):
    # distance between latitudes and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
 
    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
 
    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

In [19]:
df1['distance']=df1.apply(lambda row:haversine(row['pickup_latitude'],row['pickup_longitude'],row['dropoff_latitude'],row['dropoff_longitude']),axis=1)
df1['distance']=df1['distance'].astype(float)

In [20]:
df1.drop('pickup_longitude',inplace=True,axis=1)
df1.drop('pickup_latitude',inplace=True,axis=1)
df1.drop('dropoff_longitude',inplace=True,axis=1)
df1.drop('dropoff_latitude',inplace=True,axis=1)
df1.drop('pickup_datetime',inplace=True,axis=1)

In [21]:
df2['distance']=df2.apply(lambda row:haversine(row['pickup_latitude'],row['pickup_longitude'],row['dropoff_latitude'],row['dropoff_longitude']),axis=1)
df2['distance']=df2['distance'].astype(float)

In [22]:
df2.drop('pickup_longitude',inplace=True,axis=1)
df2.drop('pickup_latitude',inplace=True,axis=1)
df2.drop('dropoff_longitude',inplace=True,axis=1)
df2.drop('dropoff_latitude',inplace=True,axis=1)
df2.drop('pickup_datetime',inplace=True,axis=1)

In [23]:
x=df1[['vendor_id','passenger_count','store_and_fwd_flag','day_of_week','hour_of_the_day','month','distance']]
x

,vendor_id,passenger_count,store_and_fwd_flag,day_of_week,hour_of_the_day,month,distance
0,2,1,0,1,17,3,1.498521
1,1,1,0,7,0,6,1.805507
2,2,1,0,2,11,1,6.385098
3,2,1,0,3,19,4,1.485498
4,2,1,0,6,13,3,1.188588
...,...,...,...,...,...,...,...
1458639,2,4,0,5,13,4,1.225080
1458640,1,1,0,7,7,1,6.049836
1458641,2,1,0,5,6,4,7.824606
1458642,1,1,0,2,15,1,1.092564


In [24]:
y=df1[['trip_duration']]
y

,trip_duration
0,455
1,663
2,2124
3,429
4,435
...,...
1458639,778
1458640,655
1458641,764
1458642,373


In [25]:
df1

,id,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,day_of_week,hour_of_the_day,month,distance
0,id2875421,2,1,0,455,1,17,3,1.498521
1,id2377394,1,1,0,663,7,0,6,1.805507
2,id3858529,2,1,0,2124,2,11,1,6.385098
3,id3504673,2,1,0,429,3,19,4,1.485498
4,id2181028,2,1,0,435,6,13,3,1.188588
...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2,4,0,778,5,13,4,1.225080
1458640,id1049543,1,1,0,655,7,7,1,6.049836
1458641,id2304944,2,1,0,764,5,6,4,7.824606
1458642,id2714485,1,1,0,373,2,15,1,1.092564


In [26]:
df1.drop(columns=['id'], inplace=True)
idCol = df2.id.to_numpy()
df2.drop(columns=['id'], inplace=True)
yTrain = df1.trip_duration
df1.drop(columns=['trip_duration'], inplace=True)
xTest = df2.to_numpy()
xTrain = df1.to_numpy()
xTrain.shape, yTrain.shape, xTest.shape

((1458640, 7), (1458640,), (625134, 7))

In [27]:
def SMAPE(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [28]:
from lightgbm import LGBMRegressor

lgbmRegressor = LGBMRegressor()
lgbmRegressor.fit(xTrain, yTrain)
lgbmError = SMAPE(yTrain, lgbmRegressor.predict(xTrain))
lgbmError

37.48305182760548

In [29]:
final = pd.DataFrame(columns=["id","trip_duration"])
final["id"] = idCol
final["trip_duration"] = lgbmRegressor.predict(xTest)
final

,id,trip_duration
0,id3004672,736.789796
1,id3505355,736.789796
2,id1217141,413.915086
3,id2150126,1415.772126
4,id1598245,339.793751
...,...,...
625129,id3008929,322.694807
625130,id3700764,1153.829209
625131,id2568735,1347.435589
625132,id1384355,1947.868661


In [30]:
final.to_csv('submission.csv', index=False)
print('Submission succesful!')

Submission succesful!


x,y=df1.drop('Transported_True',axis=1),df1[['Transported_True']]


...
# load dataset
dataframe = pd.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

...
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(20, input_shape=(7,), kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=1)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, x, y, cv=kfold, scoring='neg_mean_squared_error')
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, x, y, cv=kfold, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))